## Reinforcement Learning with Value Iteration

These are the same maps from Module 1 but the "physics" of the world have changed. In Module 1, the world was deterministic. When the agent moved "south", it went "south". When it moved "east", it went "east". Now, the agent only succeeds in going where it wants to go *sometimes*. There is a probability distribution over the possible states so that when the agent moves "south", there is a small probability that it will go "east", "north", or "west" instead and have to move from there.

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in video games that use formal planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty in the actions. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do in **every state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

We have the same costs as before. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [1]:
costs = {".": -1, "*": -3, "^": -5, "~": -7, "x": -100000}
costs

{'.': -1, '*': -3, '^': -5, '~': -7, 'x': -100000}

and a list of offsets for `cardinal_moves`. You'll need to work this into your **actions**, A, parameter.

In [2]:
cardinal_moves = [(0, -1), (1, 0), (0, 1), (-1, 0)]

In [3]:
reward = 100000

For Value Iteration, we require knowledge of the *transition* function, as a probability distribution.

The transition function, T, for this problem is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if the agent selects "north" then 70% of the time, it will go "north" but 10% of the time it will go "east", 10% of the time it will go "west", and 10% of the time it will go "south". If agent is at the edge of the map, it simply bounces back to the current state.

You need to implement `value_iteration()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different than A\* search which only returns actions to take from the start to the goal. This also explains why reinforcement learning doesn't take a `start` state.

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Use "x" for any mountain or other impassable square. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output). Please print it out exactly as requested: **NO EXTRA SPACES OR LINES**.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy? What are the problems with this policy as it is?


In [4]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy
import numpy as np
from pprint import pprint

In [5]:
def read_world(filename):
    result = []
    with open(filename) as f:
        for line in f.readlines():
            if len(line) > 0:
                result.append(list(line.strip()))
    return result

---

<a id="value_iteration"></a>
### value_iteration

`value_iteration` performs the stochastic version of the value iteration reinforcement learning algorithm. **Uses**: [transition_sum](#transition_sum), [transition](#transition), [reward_calc](#reward_calc), [initiate_q](#initiate_q), [initiate_space](#initiate_space)

* **world** List[List]: 2-D list of spaces representing a world
* **costs** Dict: dictionary of the cost to move through any tile
* **goal** Tuple: the coordinates of the goal
* **rewards** int: the reward for getting to the goal
* **actions** List[Tuple]: list of actions allowed
* **gamma** float: the discount rate 

**return**: Dict: the dictionary of the policy found for each state

In [6]:
def value_iteration(world, costs, goal, rewards, actions, gamma):
    goal = (goal[1], goal[0])
    v_s = initiate_space(world)
    q = np.asarray(initiate_q(world, actions))
    r = reward_calc(world, costs, goal, rewards)
    tran = transition(actions)
    policy = {}
    while True:
        v_s_last = deepcopy(v_s)
        for s1, v1 in enumerate(world):
            for s2, v2 in enumerate(v1):
                for a, value in enumerate(actions):
                    t_sum = transition_sum(v_s_last, s1, s2, tran, world, actions, a)
                    if s1 == goal[0] and s2 == goal[1]:
                        q[a][s1][s2] = r[s1][s2]
                    elif world[s1][s2] == "x":
                        q[a][s1][s2] = r[s1][s2]
                    else:
                        q[a][s1][s2] = r[s1][s2] + gamma * t_sum
                max_value = -100
                for i_q, v_q in enumerate(q):
                    if q[i_q][s1][s2] > max_value:
                        max_value = q[i_q][s1][s2]
                        p = i_q
                        ref = (i_q, s1, s2)
                    if q[i_q][s1][s2] < -100:
                        p = "x"
                policy[(s1, s2)] = p
                v_s[s1][s2] = q[ref]
        epsilon = np.max(
            [i1 - i2 for i1, i2 in zip(np.asarray(v_s), np.asarray(v_s_last))]
        )
        if epsilon < 0.001:
            break
    return policy

<a id="transition_sum"></a>
### transition_sum

`transition_sum` performs the summation of the potential transitions to different states. Used in the equation to update Q. **Used by**:[value_iteration](#value_iteration)

* **v_s_last** List[List]: the list holding the last highest value action for each state
* **s1** int: the x coordinate of the current state
* **s2** int: the y coordinate of the current state
* **tran** List[List]: the chance of a transiton to a different state from a specific action
* **world** List[List]: 2-D list of spaces representing a world
* **actions** List[Tuple]: list of actions allowed 
* **a** int: the index of the current action from the list

**return**: float: the sum of the products of transitions to different states due to the current action

In [7]:
def transition_sum(v_s_last, s1, s2, tran, world, actions, a):
    v_s_last_t_list = []
    for a1, v in enumerate(actions):
        if world[s1][s2] == "x":
            v_s_last_t_list.extend([v_s_last[s1][s2]])
        elif s1 == 0 and actions[a1][0] < 0:
            v_s_last_t_list.extend([v_s_last[s1][s2]])
        elif s1 == len(world) - 1 and actions[a1][0] > 0:
            v_s_last_t_list.extend([v_s_last[s1][s2]])
        elif s2 == 0 and actions[a1][1] < 0:
            v_s_last_t_list.extend([v_s_last[s1][s2]])
        elif s2 == len(world[0]) - 1 and actions[a1][1] > 0:
            v_s_last_t_list.extend([v_s_last[s1][s2]])
        else:
            v_s_last_t_list.extend([v_s_last[s1 + actions[a1][0]][s2 + actions[a1][1]]])

    t_sum = sum([i1 * i2 for i1, i2 in zip(tran[a], v_s_last_t_list)])
    return t_sum

<a id="initiate_space"></a>
### initiate_space

`initiate_space` creates a list of lists the same size as the world with all values initialized to 0. **Used by**:[value_iteration](#value_iteration), [initiate_q](#initiate_q), [reward_calc](#reward_calc)

* **world** List[List]: 2-D list of spaces representing a world

**return**: List[List]: an empty space the same size as the word

In [8]:
def initiate_space(world):
    v_s = []
    for i, v in enumerate(world):
        temp = [0.0] * (len(world[0]))
        v_s.append(temp)
    return v_s

<a id="initiate_q"></a>
### initiate_q

`initiate_q` creates a list of lists the same size as the world with all values initialized to 0 for each available action. **Used by**:[value_iteration](#value_iteration) **Uses**:[initiate_space](#initiate_space)

* **world** List[List]: 2-D list of spaces representing a world
* **actions** List[Tuple]: list of actions allowed 

**return**: List[List]: an empty space the same size as the word

In [9]:
def initiate_q(world, actions):
    q = []
    for i, a in enumerate(actions):
        q.append(initiate_space(world))
    return q

<a id="reward_calc"></a>
### reward_calc

`reward_calc` creates a static list of lists of rewards for each space in the world. **Used by**:[value_iteration](#value_iteration) **Uses**:[initiate_space](#initiate_space)

* **world** List[List]: 2-D list of spaces representing a world
* **costs** Dict: dictionary of the cost to move through any tile
* **goal** Tuple: the coordinates of the goal
* **rewards** int: the reward for getting to the goal

**return**: List[List]: a list of rewards for moving through each state

In [10]:
def reward_calc(world, costs, goal, rewards):
    r = initiate_space(world)
    for i1, s1 in enumerate(world):
        for i2, s2 in enumerate(s1):
            r[i1][i2] = costs[s2]
            if (i1, i2) == goal:
                r[i1][i2] = rewards
    return r

<a id="transition"></a>
### transition

`transition` the list representing the chance of a transiton to a different state from a specific action. **Used by**:[value_iteration](#value_iteration)

* **actions** List[Tuple]: list of actions allowed 

**return**: List[List]: a list of the transition chance to a different state. 

In [11]:
def transition(actions):
    t = []
    tran = []
    for i, a in enumerate(actions):
        t.append([0.1] * (len(actions)))
    for i, a in enumerate(actions):
        t[i][i] = 0.7
    return t

<a id="pretty_print_policy"></a>
### pretty_print_policy

`pretty_print_policy` prints the resulting policy for each state 

* **policy** Dict: the policy for each x,y coordinate 
* **goal** Tuple: the coordinates of the goal

**return**: None 

In [12]:
def pretty_print_policy(cols, rows, policy, goal):
    r = 0
    c = 0
    p = []
    while r < rows:
        while c < cols:
            if policy[(r, c)] == 0:
                p.extend("<")
            if policy[(r, c)] == 1:
                p.extend("v")
            if policy[(r, c)] == 2:
                p.extend(">")
            if policy[(r, c)] == 3:
                p.extend("^")
            if r == goal[1] and c == goal[0]:
                p[c] = "G"
            if policy[(r, c)] == "x":
                p.extend("x")
            c = c + 1

        print("".join(p))
        p = []
        c = 0
        r = r + 1
    pass

## Value Iteration

### Small World

In [13]:
small_world = read_world("data/small.txt")

In [14]:
goal = (len(small_world[0]) - 1, len(small_world) - 1)
gamma = 0.9

small_policy = value_iteration(small_world, costs, goal, reward, cardinal_moves, gamma)

In [15]:
cols = len(small_world[0])
rows = len(small_world)
pretty_print_policy(cols, rows, small_policy, goal)

v>>>vv
vvv>vv
vvv>vv
vvvxvv
v>vvvv
>>>>vv
>>>>>G


### Large World

In [16]:
large_world = read_world("data/large.txt")

In [17]:
goal = (len(large_world[0]) - 1, len(large_world) - 1)
gamma = 0.9

large_policy = value_iteration(large_world, costs, goal, reward, cardinal_moves, gamma)

In [18]:
cols = len(large_world[0])
rows = len(large_world)
pretty_print_policy(cols, rows, large_policy, goal)

v>>>>>>>>>>>>>>vvvv>>>>>>>v
v>>>>>>>>>>>>>>vvvxxxxxxxvv
vvv^xx>>>>>>>>>vvxxxvvvxxvv
vvv<vxxxv>>>>>>>vvvvvvvxxvv
vvvvvxxvv>>>>>>>>>vvvvxxxvv
vvvvxxvvvv>>>>>>>>>>vvvxvvv
vvvxx>>vvvvvxxx>>>>>>vvvvvv
vvvv>>vvvvvvvvxxx>>>>>>vvvv
vv>>>>vvvvvvvvxx>>>>>>>>vvv
>>>>>>>vvvvxxxx>>>>>>>>>vvv
>>>>>>vvvvxxxv>>>>vvxxx>vvv
>>>>>>vvvxxvv>>>>>vvvxxvvvv
>>>>>>vvvxxvv>>>>>>>vx>>vvv
>>>>>>>vvvvvv>>>>>>>>>>>vvv
v>>^x>>>vvvvvv>>>v>>>^x>vvv
vvvxxx>>>vvvxxxv>vvv^xxvvvv
vvxx>>>>>>>vvvxxxvvxxxvvvvv
vvvxx>>>>>>>>vvvxxxxvvvvvvv
vvvxxx>>>>>>>>>vvvvvvvvvvvv
vvvvxxx>>>>>>>>>>>v>>>vvvvv
vvvvvvxx>>>>^^x>>>>>>>vvvvv
v>>vvvvxxx^^xx>>>>>>>>>vvvv
>>>>>vvvvxxxx>>>>>>>>>>>vvv
>>>>>>vvvvv>>>>^^xx>>>>>vvv
vx>>>>>>>vvxxx^^xxvxx>>>>vv
vxxx>>>>>>vvxxxxv>vvxxx>>>v
>>>>>>>>>>>>>>>>>>>>>>>>>>G
